<img src = "https://drive.google.com/uc?id=13Un3zyjk1feC8ryZvIXe-kyvOIf5FP5h">


In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np

# HopSkipJumpAttack algorithm

In [ ]:
!pip install tqdm
from tqdm import tqdm, trange
def hsja(predict, 
	sample, 
	clip_max = 1, 
	clip_min = 0, 
	constraint = 'l2', 
	num_iterations = 40, 
	gamma = 1.0, 
	target_label = None, 
	target_image = None, 
	stepsize_search = 'geometric_progression', 
	max_num_evals = 1e4,
	init_num_evals = 100,
	verbose = False):
	"""
	Main algorithm for HopSkipJumpAttack.
	Inputs:
	model: the object that has predict method. 
	predict outputs probability scores.
	clip_max: upper bound of the image.
	clip_min: lower bound of the image.
	constraint: choose between [l2, linf].
	num_iterations: number of iterations.
	gamma: used to set binary search threshold theta. The binary search 
	threshold theta is gamma / d^{3/2} for l2 attack and gamma / d^2 for 
	linf attack.
	target_label: integer or None for nontargeted attack.
	target_image: an array with the same size as sample, or None. 
	stepsize_search: choose between 'geometric_progression', 'grid_search'.
	max_num_evals: maximum number of evaluations for estimating gradient (for each iteration). 
	This is not the total number of model evaluations for the entire algorithm, you need to 
	set a counter of model evaluations by yourself to get that. To increase the total number 
	of model evaluations, set a larger num_iterations. 
	init_num_evals: initial number of evaluations for estimating gradient.
	Output:
	perturbed image.
	
	"""

	# Set parameters
	original_label = np.argmax(predict(sample)) 
  
	params = {'clip_max': clip_max, 'clip_min': clip_min, 
				'shape': sample.shape,
				'original_label': original_label, 
				'target_label': target_label,
				'target_image': target_image, 
				'constraint': constraint,
				'num_iterations': num_iterations, 
				'gamma': gamma, 
				'd': int(np.prod(sample.shape)), 
				'stepsize_search': stepsize_search,
				'max_num_evals': max_num_evals,
				'init_num_evals': init_num_evals,
				'verbose': verbose,
				}
	# Set binary search threshold.
	if params['constraint'] == 'l2':
		params['theta'] = params['gamma'] / (np.sqrt(params['d']) * params['d'])
	else:
		params['theta'] = params['gamma'] / (params['d'] ** 2)
		
	# Initialize.
	                                      
	perturbed = initialize(predict, sample, params)
	                           
	

	# Project the initialization to the boundary.
	                           
	perturbed, dist_post_update = binary_search_batch(sample, 
		np.expand_dims(perturbed, 0), 
		predict, 
		params)
	                           
	
	dist = compute_distance(perturbed, sample, constraint)
	                           

	for j in trange(params['num_iterations'], desc="HopSkipJumpattack - iterations", disable=True):
		params['cur_iter'] = j + 1

		# Choose delta.
		                           
		delta = select_delta(params, dist_post_update)
		                           

		# Choose number of evaluations.
		num_evals = int(params['init_num_evals'] * np.sqrt(j+1))
		num_evals = int(min([num_evals, params['max_num_evals']]))

		# approximate gradient.
		                           
		gradf = approximate_gradient(predict, perturbed, num_evals, 
			delta, params)
		                           
	
		if params['constraint'] == 'linf':
			update = np.sign(gradf)
		else:
			update = gradf

		# search step size.
		if params['stepsize_search'] == 'geometric_progression':
			# find step size.
			                           				
			epsilon = geometric_progression_for_stepsize(perturbed, 
				update, dist, predict, params)
			                           		
			# Update the sample. 
			perturbed = clip_image(perturbed + epsilon * update, 
				clip_min, clip_max)
			                           		
			# Binary search to return to the boundary. 
			perturbed, dist_post_update = binary_search_batch(sample, 
				perturbed[None], predict, params)

		elif params['stepsize_search'] == 'grid_search':
			# Grid search for stepsize.
			epsilons = np.logspace(-4, 0, num=20, endpoint = True) * dist
			epsilons_shape = [20] + len(params['shape']) * [1]
			perturbeds = perturbed + epsilons.reshape(epsilons_shape) * update
			perturbeds = clip_image(perturbeds, params['clip_min'], params['clip_max'])
			idx_perturbed = decision_function(predict, perturbeds, params)

			if np.sum(idx_perturbed) > 0:
				# Select the perturbation that yields the minimum distance # after binary search.
				perturbed, dist_post_update = binary_search_batch(sample, 
					perturbeds[idx_perturbed], predict, params)

		# compute new distance.
		dist = compute_distance(perturbed, sample, constraint)
		if verbose:
			print('iteration: {:d}, {:s} distance {:.4E}'.format(j+1, constraint, dist))

	return perturbed


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def decision_function(predict, images, params):
	"""
	Decision function output 1 on the desired side of the boundary,
	0 otherwise.
	"""
	                           
	images = clip_image(images, params['clip_min'], params['clip_max'])
	                           
	prob = predict(images)

	if params['target_label'] is None:
		                           
		return np.argmax(prob) != params['original_label'] 

	else:

		return np.argmax(prob) == params['target_label']





In [ ]:
def clip_image(image, clip_min, clip_max):
	# Clip an image, or an image batch, with upper and lower threshold.
	return np.minimum(np.maximum(clip_min, image), clip_max) 


In [ ]:
def compute_distance(x_ori, x_pert, constraint = 'l2'):
	# Compute the distance between two images.
	if constraint == 'l2':
		return np.linalg.norm(x_ori - x_pert)
	elif constraint == 'linf':
		return np.max(abs(x_ori - x_pert))

In [ ]:
def approximate_gradient(predict, sample, num_evals, delta, params):
	                           
	clip_max, clip_min = params['clip_max'], params['clip_min']
	                           
	# Generate random vectors.
	noise_shape =  list(params['shape'])
	                           
	if params['constraint'] == 'l2':
		rv = np.random.randn(*noise_shape)
		                           
	elif params['constraint'] == 'linf':
		rv = np.random.uniform(low = -1, high = 1, size = noise_shape)
		                           

	rv = rv / np.sqrt(np.sum(rv ** 2, axis = (0, 1), keepdims = True))
	                           
	perturbed = sample + delta * rv
	                           
	perturbed = clip_image(perturbed, clip_min, clip_max)
	                           
	rv = (perturbed - sample) / delta
	                           

	# query the model.
	decisions = decision_function(predict, perturbed, params)
	                           
	#decision_shape = [len(decisions)] + [1] * len(params['shape'])
	decision_shape = [1] + [1] * len(params['shape'])
	                           
	fval = 2 * decisions.astype(float).reshape(decision_shape) - 1.0
	                           

	# Baseline subtraction (when fval differs)
	if np.mean(fval) == 1.0: # label changes.
		                            
		gradf = np.mean(rv, axis = 0)
	elif np.mean(fval) == -1.0: # label not change.
		gradf = - np.mean(rv, axis = 0)
		                            
	else:
		fval -= np.mean(fval)
		gradf = np.mean(fval * rv, axis = 0) 
		                            

	# Get the gradient direction.
	gradf = gradf / np.linalg.norm(gradf)
	                           

	return gradf


In [ ]:
def project(original_image, perturbed_images, alphas, params):
	alphas_shape = [len(alphas)] + [1] * len(params['shape'])
	alphas = alphas.reshape(alphas_shape)
	if params['constraint'] == 'l2':
		return (1-alphas) * original_image + alphas * perturbed_images
	elif params['constraint'] == 'linf':
		out_images = clip_image(
			perturbed_images, 
			original_image - alphas, 
			original_image + alphas
			)
		return out_images

In [ ]:

def binary_search_batch(original_image, perturbed_images, predict, params):
	""" Binary search to approach the boundar. """

	# Compute distance between each of perturbed image and original image.
	dists_post_update = np.array([
			compute_distance(
				original_image, 
				perturbed_image, 
				params['constraint']
			) 
			for perturbed_image in perturbed_images])

	# Choose upper thresholds in binary searchs based on constraint.
	if params['constraint'] == 'linf':
		highs = dists_post_update
		# Stopping criteria.
		thresholds = np.minimum(dists_post_update * params['theta'], params['theta'])
	else:
		highs = np.ones(len(perturbed_images))
		thresholds = params['theta']

	lows = np.zeros(len(perturbed_images))

	

	# Call recursive function. 
	while np.max((highs - lows) / thresholds) > 1:
		# projection to mids.
		mids = (highs + lows) / 2.0
		mid_images = project(original_image, perturbed_images, mids, params)

		# Update highs and lows based on model decisions.
		decisions = decision_function(predict, mid_images, params)
		lows = np.where(decisions == 0, mids, lows)
		highs = np.where(decisions == 1, mids, highs)

	out_images = project(original_image, perturbed_images, highs, params)

	# Compute distance of the output image to select the best choice. 
	# (only used when stepsize_search is grid_search.)
	dists = np.array([
		compute_distance(
			original_image, 
			out_image, 
			params['constraint']
		) 
		for out_image in out_images])
	idx = np.argmin(dists)

	dist = dists_post_update[idx]
	out_image = out_images[idx]
	return out_image, dist

In [ ]:

def initialize(predict, sample, params):
	""" 
	Efficient Implementation of BlendedUniformNoiseAttack in Foolbox.
	"""
	success = 0
	num_evals = 0

	if params['target_image'] is None:
		# Find a misclassified random noise.
		while True:
			random_noise = np.random.uniform(params['clip_min'], 
				params['clip_max'], size = params['shape'])

			success = decision_function(predict,random_noise[None], params)
			#success = decision_function(predict,random_noise[None], params)[0]
			num_evals += 1
			if success:
				break
			assert num_evals < 1e4,"Initialization failed! "
			"Use a misclassified image as `target_image`" 


		# Binary search to minimize l2 distance to original image.
		low = 0.0
		high = 1.0
		while high - low > 0.001:
			mid = (high + low) / 2.0
			blended = (1 - mid) * sample + mid * random_noise 
			success = decision_function(predict, blended[None], params)
			if success:
				high = mid
			else:
				low = mid

		initialization = (1 - high) * sample + high * random_noise 

	else:
		initialization = params['target_image']
	return initialization

In [ ]:
def geometric_progression_for_stepsize(x, update, dist, predict, params):
	"""
	Geometric progression to search for stepsize.
	Keep decreasing stepsize by half until reaching 
	the desired side of the boundary,
	"""
	epsilon = dist / np.sqrt(params['cur_iter']) 

	def phi(epsilon):
		new = x + epsilon * update
		success = decision_function(predict, new[None], params)
		return success

	while not phi(epsilon):
		epsilon /= 2.0

	return epsilon


In [ ]:
def select_delta(params, dist_post_update):
	""" 
	Choose the delta at the scale of distance 
	between x and perturbed sample. 
	"""
	if params['cur_iter'] == 1:
		delta = 0.1 * (params['clip_max'] - params['clip_min'])
	else:
		if params['constraint'] == 'l2':
			delta = np.sqrt(params['d']) * params['theta'] * dist_post_update
		elif params['constraint'] == 'linf':
			delta = params['d'] * params['theta'] * dist_post_update	

	return delta



```
# Ce texte est au format code
```

# Loading the network ids NIDS
## You can doawnload [model500Path]( https://drive.google.com/drive/folders/1tYagVDNPgiAgBZeBOO8eODuT0FKzpC9l?usp=sharing) and [modelPath](model500Path) and save them on Google Drive 



In [1]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf

model500Path = "hidden link"
modelPath = "hidden link"

model = tf.saved_model.load(modelPath)
model500 = tf.saved_model.load(model500Path)

MessageError: ignored

# Loading attack data od NSL-KDD
## You can download the attack file [here](https://drive.google.com/file/d/1h1Q89-l9_JC9R-XPNpxdSSwJLzT2w4QL/view?usp=sharing)


In [ ]:
from numpy import load
#xTrainPath = 'hidden link'
#yTrainPath = 'hidden link'
AttacksPath = 'hidden path'
Attacks = load(AttacksPath)
#Attacks = np.reshape(Attacks, (Attacks.shape[0], 113))
#xTrain = load(xTrainPath, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
#yTrain = load(yTrainPath)

# Prediction function of the NIDS

In [ ]:
import numpy as np
def predictx(x):
  inp = tf.convert_to_tensor(x)
  inp = tf.cast(inp, tf.float32)
  #if len(inp.shape) > 3:
    #inp = np.expand_dims(inp, 0) 
  inp = np.reshape(inp, (1, 1, 113))
  labeling = model500(inp)
  prediction = np.argmax(labeling)
  oneHotEncode = [0, 0]
  if prediction == 0:
    oneHotEncode[0] = 1
  else:
    oneHotEncode[1] = 1
  return np.array(oneHotEncode)

# AE generation with HSJA attack

In [ ]:
# Generate an adversarial example
# The parameters
adv_examples = []
predict = predictx
#sample = xTrain[2]
clip_max = 5
clip_min = -5
constraint = 'l2'
num_iterations = 40
gamma = 1.0
target_label = 0
target_image = None, 
stepsize_search = 'geometric_progression', 
max_num_evals = 1e4
init_num_evals = 100
for i in trange(100, desc="HopSkipJumpAttack - iterations", disable=False):
  sample = Attacks[i]
  adv = hsja(predictx, sample, clip_max, clip_min, constraint, num_iterations, gamma, target_label, None, stepsize_search, max_num_evals, init_num_evals, False)
  adv_examples.append(adv)

HopSkipJumpAttack - iterations: 100%|██████████| 100/100 [01:00<00:00,  1.65it/s]


# Saving the AE in Google Drive

In [ ]:
f = "hidden path"
np.save(f, adv_examples)

# Attacking the NIDS

In [ ]:
success = 0
def predict(model, x):
  inp = tf.convert_to_tensor(x)
  inp = tf.cast(inp, tf.float32)
  inp = tf.reshape(inp, ( 1, 1, 113))
  labeling = model(inp)
  label = np.argmax(labeling)
  
  return label
for i in range(100):
  prediction = predict(model500,adv_examples[i])
  if prediction == 0:
    success += 1



In [ ]:
print("success rate of HSJA is ", success, "%")


success rate of HSJA is  100 %
